# Deploy your Custom Model using Internal WML CPD

## 1. Setup the Notebook Environment

## 1.1 Install the necessary packages

### Scikit-learn version 0.20

In [2]:
!pip install scikit-learn==0.20.3

### Watson Machine Learning Python SDK

In [3]:
!pip install watson-machine-learning-client -U

     |################################| 542kB 10.5MB/s eta 0:00:01


Restart the Notebook after Installing the required packages. By clicking on `Kernel>Restart`

## 1.2 Import Packages

In [1]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn import preprocessing
from sklearn import svm, metrics
from scipy import sparse
from watson_machine_learning_client import WatsonMachineLearningAPIClient
from sklearn.preprocessing import StandardScaler, LabelEncoder, Imputer, OneHotEncoder
import json
import ibm_db


import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV



2020-06-04 07:13:31,148 - watson_machine_learning_client.wml_client_error - WARNING - It is mandatory is set the space or project. Use client.set.default_space(<SPACE_GUID>) to set the space or client.set.default_project(<PROJECT_GUID>).


## 2. Configuration

### 2.1 Global Variables

Check the documentation- https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_current/wsj/analyze-data/ml-spaces_local.html on steps to create a deployment space

In [18]:
MODEL_NAME="TOWER_A"
DEPLOYMENT_NAME="TOWER_A"

dep_name='Telco_Deployment'

### 2.2 Add Dataset

Select the `Insert Pandas Dataframe` option, after selecting the below cell. Ensure the variable name is `df_data_1`

In [19]:

df_data_1 = pd.read_csv('/project_data/data_asset/call_drop_data_train.csv')
df_data_1.head()


,outgoing_site_id,Start_Time_MM_DD_YYYY,Start_Time_HH_MM_SS_s,Call_Service_Duration,_conds,_dewptm,_fog,_hail,_hum,_pressurem,total number_of_calls,total call duration (min),Traffic,lat,long,Call_Drop_Count,Total_Calls,Call_Drop_Perc,Call_Drop_Actual
0,63383,01/01/17,00:00 to 4:00,325,Smoke,11,0,0,72,1019.0,631,455,High,12.972967,77.609317,11100,12234,0.907308,0
1,63383,01/01/17,4:00 to 8:00,424,Shallow Fog,11,1,0,82,1019.0,540,239,High,12.972967,77.609317,11100,12234,0.907308,0
2,63383,01/01/17,8:00 to 12:00,161,Shallow Fog,11,1,0,88,1020.0,423,309,Medium,12.972967,77.609317,11100,12234,0.907308,0
3,63383,01/01/17,12:00 to 16:00,383,Fog,11,1,0,100,1020.0,650,428,High,12.972967,77.609317,11100,12234,0.907308,0
4,63383,01/01/17,16:00 to 20:00,330,Fog,8,1,0,93,1019.0,240,443,Low,12.972967,77.609317,11100,12234,0.907308,0


### 2.3 Add your WML Credentials

In [20]:
import sys,os,os.path


WML_CREDENTIALS = {
"token": os.environ['USER_ACCESS_TOKEN'],
"instance_id" : "wml_local",
"url" : os.environ['RUNTIME_ENV_APSX_URL'],
"version": "2.5.0"
}

### 2.4 Update your AIOS Credentials

In [21]:
WOS_CREDENTIALS={
    "url" : os.environ['RUNTIME_ENV_APSX_URL'],
    "username":"admin",
    "password":"password"
}

### 2.5 Add your Db credentials

#### These Db credentials are needed ONLY if you have NOT configured your `OpenScale Datamart`. 

Ensure the name  of the variable as- `DATABASE_CREDENTIALS` 

In [22]:

# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.
from project_lib import Project
project = Project.access()

DATABASE_CREDENTIALS = project.get_connection(name="Smruthi Db2 on Cloud")


## 3. Create the Call Drop Model using Scikit-Learn

In [23]:
X=df_data_1.drop(['Call_Drop_Actual'], axis=1)
y=df_data_1.loc[:, 'Call_Drop_Actual']

In [24]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)



In [25]:
'''Add a categorical transformer to your model pipeline. 
    You will need to add a label encoder into the model pipeline before storing it into WML '''

categorical_features = ["Start_Time_MM_DD_YYYY", "Traffic", " _conds", "Start_Time_HH_MM_SS_s"]
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [26]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

In [27]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', svm.SVC(kernel='linear'))])

In [28]:
model = pipeline.fit(X_train,y_train)

In [29]:
client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [30]:
meta_props={
 client.repository.ModelMetaNames.NAME: "Call Drop Prediction model",
 client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3.6",
 client.repository.ModelMetaNames.TYPE: "scikit-learn_0.20",
}

## 4. Create a new Deployment Space



In [31]:
project_id = os.environ['PROJECT_ID']
client.set.default_project(project_id)

'SUCCESS'

In [32]:
def guid_from_space_name(client, space_name):

    instance_details = client.service_instance.get_details()

    space = client.spaces.get_details()
    res=[]
    for item in space['resources']: 
        if item['entity']["name"] == space_name:
            res=item['metadata']['guid']

    return res

In [33]:
# Enter the name of your deployment space of the current project

space_uid = guid_from_space_name(client, dep_name)

In [34]:
space_uid

'279451d8-efb0-40b6-80de-a50ebed94ff1'

In [35]:
client.set.default_space(space_uid)

Unsetting the project_id ...


'SUCCESS'

In [36]:
 deploy_meta = {
     client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
     client.deployments.ConfigurationMetaNames.ONLINE: {}
 }

In [37]:
## Store the model on WML
published_model = client.repository.store_model(pipeline,
                                             meta_props=meta_props,
                                             training_data=X_train,
                                             training_target=y_train
                                                )





In [40]:
published_model_uid = client.repository.get_model_uid(published_model)

In [41]:
## Create a Deployment for your stored model

created_deployment = client.deployments.create(published_model_uid, meta_props=deploy_meta)





#######################################################################################

Synchronous deployment creation for uid: '8c32d355-6745-4492-9677-a7515cd4946c' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='bc1030d1-8a33-4a6a-9dff-4cd738465aa2'
------------------------------------------------------------------------------------------------




In [42]:
client.deployments.get_details()

{'resources': [{'metadata': {'parent': {'href': ''},
    'guid': 'bc1030d1-8a33-4a6a-9dff-4cd738465aa2',
    'modified_at': '',
    'created_at': '2020-06-04T07:33:02.341Z',
    'href': '/v4/deployments/bc1030d1-8a33-4a6a-9dff-4cd738465aa2'},
   'entity': {'name': 'TOWER_A',
    'custom': {},
    'online': {},
    'description': '',
    'space': {'href': '/v4/spaces/279451d8-efb0-40b6-80de-a50ebed94ff1'},
    'status': {'state': 'ready',
     'online_url': {'url': 'https://internal-nginx-svc:12443/v4/deployments/bc1030d1-8a33-4a6a-9dff-4cd738465aa2/predictions'}},
    'asset': {'href': '/v4/models/8c32d355-6745-4492-9677-a7515cd4946c?space_id=279451d8-efb0-40b6-80de-a50ebed94ff1'},
    'auto_redeploy': False}}]}

In [43]:

scoring_endpoint = None
deployment_uid=created_deployment['metadata']['guid']
print(deployment_uid)

for deployment in client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        scoring_endpoint = deployment['entity']['status']['online_url']['url']
        
print(scoring_endpoint)

bc1030d1-8a33-4a6a-9dff-4cd738465aa2
https://internal-nginx-svc:12443/v4/deployments/bc1030d1-8a33-4a6a-9dff-4cd738465aa2/predictions


In [57]:
score=X_test.tail(1)
score

,outgoing_site_id,Start_Time_MM_DD_YYYY,Start_Time_HH_MM_SS_s,Call_Service_Duration,_conds,_dewptm,_fog,_hail,_hum,_pressurem,total number_of_calls,total call duration (min),Traffic,lat,long,Call_Drop_Count,Total_Calls,Call_Drop_Perc
8482,63383,27/02/17,16:00 to 20:00,185,Partly Cloudy,18,0,0,83,1012.0,351,496,Medium,12.972967,77.609317,11100,12234,0.907308


In [58]:
scoring_data=list(list(x) for x in zip(*(score[x].values.tolist() for x in score.columns)))
scoring_data

[[63383,
  '27/02/17',
  '16:00 to 20:00',
  185,
  'Partly Cloudy',
  18,
  0,
  0,
  83,
  1012.0,
  351,
  496,
  'Medium',
  12.972967,
  77.609317,
  11100,
  12234,
  0.907307504]]

In [59]:
fields=list(X_test.columns)
print(len(fields))
fields, scoring_data

18


(['outgoing_site_id',
  'Start_Time_MM_DD_YYYY',
  'Start_Time_HH_MM_SS_s',
  'Call_Service_Duration',
  ' _conds',
  ' _dewptm',
  ' _fog',
  ' _hail',
  ' _hum',
  ' _pressurem',
  'total number_of_calls',
  'total call duration (min)',
  'Traffic',
  'lat',
  'long',
  'Call_Drop_Count',
  'Total_Calls',
  'Call_Drop_Perc'],
 [[63383,
   '27/02/17',
   '16:00 to 20:00',
   185,
   'Partly Cloudy',
   18,
   0,
   0,
   83,
   1012.0,
   351,
   496,
   'Medium',
   12.972967,
   77.609317,
   11100,
   12234,
   0.907307504]])

In [68]:

job_payload = {
client.deployments.ScoringMetaNames.INPUT_DATA: [{
 'values': scoring_data
}]
}
print(job_payload)

{'input_data': [{'values': [[63383, '27/02/17', '16:00 to 20:00', 185, 'Partly Cloudy', 18, 0, 0, 83, 1012.0, 351, 496, 'Medium', 12.972967, 77.609317, 11100, 12234, 0.907307504]]}]}


In [69]:
scoring_response = client.deployments.score(deployment_uid, job_payload)

print(scoring_response)

{'predictions': [{'fields': ['prediction'], 'values': [[0]]}]}
